In [1]:
import os

import dask.dataframe as dd
import pandas as pd

# How to Subset the Pre-Computed Travel Cost Matricies
To calculate spatial access measures, data on travel times or distances between origins and destinations must be used. If you only need distances between origins and destinations, the `access` package will calculate Euclidean distances for your projected data. If you need travel times for a specific travel mode (walking, public transit, or driving) you need to generate these so-called travel time (or travel cost) matrices from other sources. We provide pre-calculated travel cost matricies between blocks and tracts [here](https://access.readthedocs.io/en/latest/resources.html) for the 20 largest cities in the US and for the entire country.

Each dataset are point-to-point distance matrices generated by the Center for Spatial Data Science. The matrices come as bzipped CSVs. Cities are listed according to their containing county code. The origins for each distance matrix are the population-weighted centroids that lie within each county, and the destinations are any population-weighted centroids that lie within 100 km of the buffered county.

In this notebook, we show how to download the national travel cost matrix and subset it for your area of interest. In the example below, we will subset the matrix to include only the tracts in Cook County, IL. By the end of this notebook, you should be able to:
- [Download the travel matrix](#Downloading-the-National-Travel-Matrix)
- [Define your area of interest using the Census GEOID](#Defining-Area-of-Interest-with-Census-GEOID)
- [Use dask to subset the travel cost matrix to your area of interest](#Use-Dask-to-Subset-the-Travel-Cost-Matrix)
- [Use the terminal and basic bash commands to subset the travel cost matrix to your area of interest](#Use-Bash-Commands-to-Subset-the-Travel-Cost-Matrix)


## Downloading the National Travel Matrix
You can find the national driving travel cost matrix on the [`access` read the docs page](https://access.readthedocs.io/en/latest/resources.html). You can download the dataset either by clicking on the link as shown in the image below, or as shown in the code which follows.
<img src="../docs/_static/images/screenshot_cost_website.png" style="width: 750px;">


In [ ]:
! wget https://uchicago.box.com/shared/static/prapz7ac7vwuz44nnab3dhe10vbg55cz.bz2 \
    -O ./national_cost_matrix.csv.bz2  # noqa: E501

You should now see the file in the current directory.



In [ ]:
os.listdir("./")

## Defining Area of Interest with Census GEOID
You can find more information on the Census GEOID [here](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html). For our example, we will filter the dataset for Cook County, IL. The State ID for Illinois is `17` and the County ID for Cook County is `031`.TOgether, we want to filter our dataset for rows that have an origin and destination that start with `17031`. 

## Use Dask to Subset the Travel Cost Matrix
Since the uncompressed csv is too large to load into the memory of computers with less than 8GB of memory, we cannot load the data in its entirety into memory using `pandas`. Even if you can load it into memory, using `pandas` may take too long. Instead, we will show you how to use [`dask`](https://docs.dask.org/en/latest/delayed.html) to subset the travel cost matrix. First, we must uncompress the file and extract the csv. Note: you must install the command line package `bzip2` if it's not already installed.

In [ ]:
! bzip2 -dk national_cost_matrix.csv.bz2

Note that this is a big file, uncompressed -- more than 4GB.  You may want to delete it when you're done.

Next, we'll read in the csv dataset using dask dataframe's `.read_csv()` method. Note: nothing has yet happend by running this command. Dask uses lazy evaluation, so nothing will compute until you use the `.compute()` method.

In [ ]:
national_cost_matrix = dd.read_csv("./national_cost_matrix.csv")

# Filter out faulty rows which exist in the data - will be cleaned and replaced soon.
national_cost_matrix = national_cost_matrix[national_cost_matrix["origin"] != "origin"]

We will now convert the origin and destination columns to type `str` and use the newly converted string column to filter for `17031`.

In [ ]:
national_cost_matrix["origin"] = national_cost_matrix.origin.astype(str)
national_cost_matrix["destination"] = national_cost_matrix.destination.astype(str)

cook_county_fips = "17031"

cook_cost_matrix = national_cost_matrix[
    national_cost_matrix["origin"].str.startswith(cook_county_fips)
    & national_cost_matrix["destination"].str.startswith(cook_county_fips)
]

With our delayed transformations setup, now we can execute our data transformations and have `dask` complete those transformations in parallel with the `.compute()` method. **Warning: you might need at least 8GB of memory to successfully execute this process. If you have 8GB of memory and it fails, try closing unused programs and try again.**

In [ ]:
cook_cost_matrix = cook_cost_matrix.compute()

In [ ]:
cook_cost_matrix.head()

Make sure to save a copy of the subsetted data!

In [ ]:
cook_cost_matrix.to_csv("cook_county_cost_matrix.csv", index=False)

## Use Bash Commands to Subset the Travel Cost Matrix
You can also use a bash script to subset the national Travel Cost matrix provided you have two files, each containing the origin and destination geoids you're interested in capturing.

Here's an example where we use two files containing a small list of origin and destination locations, split into separate `origin.csv` and `destinations.csv` files.

In [2]:
orig_dest = pd.DataFrame(
    {
        "origin": ["17031010100", "17031010101", "17031010102"],
        "destination": ["17031820605", "17031292400", "17031827100"],
    }
)

In [3]:
orig_dest["origin"].to_csv("./origins.csv", index=False, header=False)

In [4]:
pd.read_csv("./origins.csv", header=None).head()

,0
0,17031010100
1,17031010101
2,17031010102


In [5]:
orig_dest["destination"].to_csv("./destinations.csv", index=False, header=False)

In [6]:
pd.read_csv("./destinations.csv", header=None).head()

,0
0,17031820605
1,17031292400
2,17031827100


In addition to the two files above, you'll also need the `r.awk` script (included in this directory) and the unzipped national Travel Cost matrix.

Below is the simple `r.awk` script.  What this does, line by line, is:

0. Keep track of the file number, by checking if it's the first line of a file.
1. For file number 1, add the first field (we've done `-F','`, so this is comma separated) to a list of origins.
2. For file number 2, add the first field to a list of destinations.
3. For other files (i.e. number 3), check to see if field one is an origin and field two is a destination.  If so, it will print.

In [7]:
!cat r.awk


FNR == 1 { file+=1 }

file == 1 { ORIGINS[$1]=1 ; next } 

file == 2 { DESTINATIONS[$1]=1 ; next } 

($1 in ORIGINS) && ($2 in DESTINATIONS)



The next cell shows, how to use the script to create your desired subset. The command follows the structure:

`awk -F',' -f ./r.awk [origins_file] [destinations_file] [travel_cost_matrix] > [subset_output]`

In [8]:
!awk -F',' -f ./r.awk ./origins.csv ./destinations.csv ./national_cost_matrix.csv \
    > subset_cost_matrix.csv

Now we can read the newly created travel matrix subset, making sure to include column names for the dataset.

In [9]:
subset_matrix = pd.read_csv(
    "./subset_cost_matrix.csv", names=["origin", "destination", "minutes"]
)

In [10]:
subset_matrix

,origin,destination,minutes
0,17031010100,17031292400,46.88
1,17031010100,17031820605,59.28
2,17031010100,17031827100,62.75
